In [1]:
import pycosat
import secrets
import math
import numpy as np

In [2]:
file = open("Scrum.dimacs")

In [3]:
lines = file.readlines()

In [4]:
names = []
for line in lines[:124]:
    names.append(line.split(' ')[2][:-1])

In [5]:
dimacs = lines[125:]

In [6]:
len(dimacs)

255

In [7]:
cnf = [[int(s)  for s in line.split(' ') if int(s) != 0] for line in dimacs]

In [8]:
solutions = []
i = 0
for sol in pycosat.itersolve(cnf):
    if i == 100000:
        break
    i += 1
    solutions.append(sol)
print(i)

100000


In [9]:
class Item:
    def __init__(self, item):
        self.r = -1
        self.d = -1
        self.theta = -1
        self.item = item
        self.features = sum(item)
        self.totalcost = secrets.randbelow(100)
        self.knowndefects = secrets.randbelow(100)
        self.featuresused = secrets.randbelow(self.features)

In [10]:
binary_solutions = [[1 if val > 0 else 0 for val in sol] for sol in solutions]

items = [Item(item) for item in binary_solutions]
max_features = -math.inf
min_features = math.inf
max_totalcost = -math.inf
min_totalcost = math.inf
max_known = -math.inf
min_known = math.inf
max_featuresused = -math.inf
min_featuresused = math.inf

for x in items:
    if x.features > max_features:
        max_features = x.features
    if x.features < min_features:
        min_features = x.features
    if x.totalcost > max_totalcost:
        max_totalcost = x.totalcost
    if x.totalcost < min_totalcost:
        min_totalcost = x.totalcost
    if x.knowndefects > max_known:
        max_known = x.knowndefects
    if x.knowndefects < min_known:
        min_known = x.knowndefects
    if x.featuresused > max_featuresused:
        max_featuresused = x.featuresused
    if x.featuresused < min_featuresused:
        min_featuresused = x.featuresused

In [11]:
total_group = 10
west = []
east = []
westItems = []
eastItems = []
rand = secrets.choice(items)
max_r = -math.inf
min_r = math.inf
for x in items:
    x.r = sum(x.item)
    x.d = sum([a_i - b_i for a_i, b_i in zip(x.item, rand.item)])
    if x.r > max_r:
        max_r = x.r
    if x.r < min_r:
        min_r = x.r
for x in items:
    x.r = (x.r - min_r)/(max_r - min_r)
R = set([r.r for r in items])
for k in R:
    g = [item for item in items if item.r == k]
    g.sort(key=lambda x: x.d, reverse=True)
    for i in range(len(g)):
        g[i].theta = (2*math.pi*(i+1))/len(g)
thk = max_r / total_group
for a in range(total_group):
    g = [i for i in items if (a*thk)<= i.r <= ((a+1)*thk)]
    #print(a)
    g.sort(key=lambda x: x.theta)
    if len(g) > 0:
        east.append(g[0])
        west.append(g[len(g)-1])
        for i in g:
            if i.theta <= math.pi:
                eastItems.append(i)
            else:
                westItems.append(i)

0
1
2
3
4
5
6
7
8
9


In [11]:
def split_bin(items, total_group):
    west = []
    east = []
    westItems = []
    eastItems = []
    rand = secrets.choice(items)
    max_r = -math.inf
    min_r = math.inf
    for x in items:
        x.r = sum(x.item)
        x.d = sum([a_i - b_i for a_i, b_i in zip(x.item, rand.item)])
        if x.r > max_r:
            max_r = x.r
        if x.r < min_r:
            min_r = x.r
    for x in items:
        x.r = (x.r - min_r)/(max_r - min_r + 10**(-32))
    R = set([r.r for r in items])
    for k in R:
        g = [item for item in items if item.r == k]
        g.sort(key=lambda x: x.d, reverse=True)
        for i in range(len(g)):
            g[i].theta = (2*math.pi*(i+1))/len(g)
    thk = max_r / total_group
    for a in range(total_group):
        g = [i for i in items if (a*thk)<= i.r <= ((a+1)*thk)]
        #print(a)
        g.sort(key=lambda x: x.theta)
        if len(g) > 0:
            east.append(g[0])
            west.append(g[len(g)-1])
            for i in g:
                if i.theta <= math.pi:
                    eastItems.append(i)
                else:
                    westItems.append(i)
    return west, east, westItems, eastItems

In [12]:
def better():
    what = 1
    return what

In [13]:
from itertools import count

class TreeNode:
    _ids = count(0)

    def __init__(self, east, west, east_node, west_node):
        self.id = next(self._ids)
        self.east = east
        self.west = west
        self.east_node = east_node
        self.west_node = west_node


In [14]:
def sway(items, enough):
    if len(items) < enough:
        return TreeNode(items, None, None, None)
    delta1, delta2 = [], []
    west, east, west_items, east_items = split_bin(items, 10)
    east_node = sway(east_items, enough)
    west_node = sway(west_items,enough)
    node = TreeNode(east, west, east_node, west_node)
    return node

In [15]:
tree = sway(items, 100)

In [16]:
difference(tree.west[0], tree.east[0])

NameError: name 'difference' is not defined

In [ ]:
def difference(west, east):
    w = np.array(west.item)
    e = np.array(east.item)
    res = np.logical_xor(w, e)
    res = np.logical_not(res)
    return (len(res) - np.sum(res))

In [ ]:
def bfs(tree, treshold):
    # maintain a queue of paths
    queue = []
    # push the first path into the queue
    queue.append([tree])
    smallest_diff = 150
    while queue:
        # get the first path from the queue
        path = queue.pop(0)
        path_id = [x.id for x in path]
        # get the last node from the path
        #import pdb; pdb.set_trace()
        node = path[-1]
        if(node.west):
            diff = difference(node.west[0], node.east[0])
            #print(diff)
            # path found
            if diff < smallest_diff:
                smallest_diff = diff
            #checks to find question.
            if diff <= treshold:
                return path_id, node
            # enumerate all adjacent nodes, construct a new path and push it into the queue
            neighbors = []
            if(node.west_node):
                neighbors.append(node.west_node)
            if(node.east_node):
                neighbors.append(node.east_node)
            for adjacent in neighbors:
                new_path = list(path)
                new_path.append(adjacent)
                queue.append(new_path)
            
        
    print("Couldnt find", smallest_diff, path_id)

In [ ]:
bfs(tree, 10)

In [ ]:
def prLevel(root): 
  
    if (not root): 
        return
  
    # queue to hold tree node with level 
    q = []  
  
    # let root node be at level 1 
    q.append([root, 1]) 
  
    p = [] 
  
    # Do level Order Traversal of tree 
    while (len(q)):  
        p = q[0] 
        q.pop(0) 
        #print("Level of", p[0].id, "is", p[1]) 
        if (p[0].west_node): 
            q.append([p[0].west_node, p[1] + 1]) 
        if (p[0].east_node): 
            q.append([p[0].east_node, p[1] + 1 ]) 

In [ ]:
prLevel(tree)

In [ ]:
def rankFeatures(items):
    count = np.zeros(len(items[0].item))
    for item in items:
        count = np.add(count, item.item)
    rank = np.zeros(len(count))
    for i, v in enumerate(count):
        if v == 0:
            rank[i] = -1
            print("No", names[i])
        if v == (len(items)):
            rank[i] = -1
            print("All", names[i])
    return count, rank

In [ ]:
len(items)

In [ ]:
count, rank = rankFeatures(items)

In [ ]:
for i, r in enumerate(rank):
    if r == 0:
        print(names[i])
print(len(rank))